In [1]:
from transformers import AutoModelWithLMHead, AutoTokenizer
from torch import tensor,argmax
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering

In [2]:
tokenizer_q = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model_q = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model_a = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer_a = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
c:\Users\sshiv\anaconda3\envs\tf-gpu\lib\site-packages\transformers\models\auto\modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [3]:
def get_questions(context, max_length=64):
    qns=[]
    sentences=context.split('.')
    for sentence in sentences[:-1]:
        input_text = "answer: %s  context: %s </s>" % ('', sentence)
        features = tokenizer_q([input_text], return_tensors='pt')

        output = model_q.generate(input_ids=features['input_ids'], 
                   attention_mask=features['attention_mask'],
                   max_length=max_length)
        qns.append(tokenizer_q.decode(output[0]).replace('<pad> question: ','').replace('</s>',''))
    return qns

In [4]:
def answer_question(question, context):
    input_ids = tokenizer_a.encode(question, context)
    sep_index = input_ids.index(tokenizer_a.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    outputs = model_a(tensor([input_ids]),
                    token_type_ids=tensor([segment_ids]),
                    return_dict=True) 
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    answer_start = argmax(start_scores)
    answer_end = argmax(end_scores)
    tokens = tokenizer_a.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]
    return answer

# Testing

## Generate Questions

In [5]:
context_q = input("Enter the Context to Generate Questions : \n\n\t")
print("\nGenerated Questions ",end='\n\n')
question_q=list(set(get_questions(context_q)))
for i,qn in enumerate(question_q,start=1):
    print(str(i)+') '+qn)


Generated Questions 

1) What is the name of the concept of machine learning?
2) How can Alexa help you listen to music?
3) What does pattern recognition do?
4) What is one of the most exciting subsets of Artificial Intelligence?
5) What is the future of Machine Learning?
6) What is the name of the branch of machine learning that is all around us?
7) What is the purpose of the prediction and detection technology?
8) What is the power of machine learning?
9) What will Alexa do when you ask her to play your favorite music station?
10) What is the current trend of automating the application of complex mathematical calculations to big data?
11) What is important to understand about Machine Learning?
12) What is the ability of machine learning to adapt to new data independently and through iterations?
13) What is the purpose of the learning process?
14) What does this help the algorithms teach the computer?


## Answering the Question

In [6]:
context_a = input("Enter the Context to Answer the Question : \n\n")
question_a = input("\nEnter the Question to Answer from the Context : \n\n")
print("\nGenerated Answer",end='\n\n')
answer_a=answer_question(question_a,context_a)
print(' '+answer_a)


Generated Answer

 machine learning


## Generate Questions and Answers

In [8]:
context_q_a = input("Enter the Context to Generate Questions and Answers : \n\n\t")
print("\nGenerated Question and Answers",end='\n\n')
question_q_a=list(set(get_questions(context_q_a)))
answer_q_a=[]
for qn in question_q_a:
    answer_q_a.append(answer_question(qn,context_q_a))
for i in range(len(question_q_a)):
    print('Qn.'+str(i+1)+'  '+question_q_a[i],sep='\n')
    print('Ans : '+answer_q_a[i],sep='\n',end='\n\n')



Generated Question and Answers

Qn.1  What is the name of the concept of machine learning?
Ans : automating the application of complex mathematical calculations to big data

Qn.2  How can Alexa help you listen to music?
Ans : she will go to the station you played most often

Qn.3  What does pattern recognition do?
Ans : produce reliable and informed results

Qn.4  What is one of the most exciting subsets of Artificial Intelligence?
Ans : machine learning

Qn.5  What is the future of Machine Learning?
Ans : how it can be used

Qn.6  What is the name of the branch of machine learning that is all around us?
Ans : machine learning

Qn.7  What is the purpose of the prediction and detection technology?
Ans : helps computer systems learn and improve from experience by developing computer programs that can automatically access data and perform tasks

Qn.8  What is the power of machine learning?
Ans : data

Qn.9  What will Alexa do when you ask her to play your favorite music station?
Ans : go

In [10]:
answer_q_a

['automating the application of complex mathematical calculations to big data',
 'she will go to the station you played most often',
 'produce reliable and informed results',
 'machine learning',
 'how it can be used',
 'machine learning',
 'helps computer systems learn and improve from experience by developing computer programs that can automatically access data and perform tasks',
 'data',
 'go to the station you played most often',
 'gaining more momentum',
 'what makes machine learning work',
 'machine learning',
 'learning from data with specific inputs to the machine',
 'as you input more data into a machine']